# Spieleprogrammierung für fortgeschrittene Programmierer

In der Jahrgangstufe 11 wurden grundlegende Prinzipien der Spieleprogrammierung mit pygame bereits behandelt:

Der prinzipielle Aufbau eines Spiels:
* Init-Sequenz
* Game-Loop mit
    * Überprüfung von Events: Maus-Klicks, Tastatureingabe
    * Spielelogik (Verändern der Objekt-Positionen, Kollisionsüberprüfung, etc.)
    * Neuzeichnen der Oberfläche

Fenster haben links oben den Punkt (0,0). Nach rechts werden die x-Werte größer, nach **unten** werden die y-Werte größer.

Im folgenden sollen ein paar weitere Konzepte exemplarisch mit einfließen:
* Konzept von Surface und Rect (Version 1)
* Verwendung von Klassen zur Kapselung und Erzeugung von mehr Übersichtlichkeit (Version 2)
* Kollisionsabfrage von Rect-Objekten (Version 3)
* Einsatz von Sprites und (Sprite-)Groups (Version 4)


# Version 1: Spielelemente als Surface und Rect - keine Klassen
Ein sehr einfaches Spiel soll realisiert werden. Es wird bewusst zunächst auf hübsche Grafiken verzichtet und stattdessen sogenannte Surface-Objekte verwendet. Später lassen sich diese Surface-Objekte auf einfache Art mit Grafiken (png, jpg, etc.) ersetzen. (Im Programmcode ist bereits angedeutet, wie das möglich ist.)

Insgesamt werden drei verschiedene "Dinge" angelegt:
* Der Boden ("Floor") mit ```floor_surf``` als Surface-Objekt und ```floor_rect``` als Rect-Objekt (Rectangular = Rechteck).
* Der Spieler ("Player") mit ```player_surf``` und ```player_rect```. Er kann mit den Pfeiltasten nach links und rechts bewegt werden.
* Ein Block ("Block") mit ```blk_surf``` und ```blk_rect```, das von oben nach unten fällt und bei Berührung des Bodens wieder oben neu erscheint.

Es wird hier jeweils erst ein einfarbiges Surface-Objekt (z.B. ```player_surf```) erstellt. Wir geben hier die Größe vor; man könnte auch eine Bilddatei importieren und darüber die (rechteckige) Größe erhalten. Die Größe lässt sich selbstverständlich anpassen. Aus dem Surface-Objekt kann dann ein Rect-Objekt erzeugt werden:

```player_rect = pygame.Surface.get_rect(player_surf)```

Das Rect-Objekt dient uns später zur Kontrolle der Position. Man kann auf viele verschiedene "Punkte" des Rects direkt zugreifen:
* ```player_rect.x``` und ```player_rect.y``` (x-/y-Position der oberen linken Ecke)
* ```player_rect.center``` (Mittelpunkt: z.B. ```player_rect.center = (20,20)```
* ```player_rect.midbottom``` (Mittelpunkt der Unterkante: z.B. ```player_rect.midbottom = (20,40)```
* Weiterhin auf: top, left, bottom, right, topleft, bottomleft, topright, bottomright, midtop, midleft, midbottom, midright, center, centerx, centery ...

In der Game-Loop wird bewusst wenig gemacht. Die ganze Logik findet in der ```update()```-Methode statt.

Das Neuzeichnen von Boden, Spieler und Block erfolgt jeweils über die ```blit()```-Methode. Diese braucht als Parameter das Aussehen (z.B. ```player_surf```) und die Position (z.B. ```player_rect```).

In [ ]:
# Schritt 1: Importieren u. initialisieren der Pygame-Bibliothek
import random
import pygame
from pygame.locals import *
pygame.init()

# Schritt 2a: Variablen/KONSTANTEN setzen
SCREEN_WIDTH  = 300
SCREEN_HEIGHT = 480
FPS  = 60

# Schritt 2b: Hilfs-Funktionen für das Hauptprogramm

# Schritt 3: Definieren und Öffnen eines neuen Fensters
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Titel für Fensterkopf")
clock = pygame.time.Clock()

# Floor (So breit wie das Fenster, Höhe 1/4 des Fensters)
floor_surf = pygame.Surface((screen.get_width(), screen.get_height()//4))
floor_surf.fill("green")
floor_rect = pygame.Surface.get_rect(floor_surf)
floor_rect.topleft = (0,screen.get_height()*3//4)  # Position: x = 0, y = 3/4 des Fensters von oben

# Player
player_surf = pygame.Surface((20,20))
player_surf.fill("black")
#player_surf = pygame.image.load("images/player.png").convert_alpha()
player_rect = pygame.Surface.get_rect(player_surf)
player_rect.midbottom = floor_rect.midtop  # Position: Mitte der Unterkante des Spieler in der Mitte vom Boden

# Block
blk_surf = pygame.Surface((20,20))
blk_surf.fill("red")
blk_rect = pygame.Surface.get_rect(blk_surf)
blk_rect.center = (random.randint(10,screen.get_width()-10), -30) # y-Position oberhalb vom sichtbaren Bereich

def update():
    # object
    blk_rect.y += 3
    if blk_rect.bottom >= floor_rect.top:
        blk_rect.center = (random.randint(10,screen.get_width()-10), -30)
    # player
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        player_rect.x -= 3
        if player_rect.left < 0:
            player_rect.left = 0
    if keys[pygame.K_RIGHT]:
        player_rect.x += 3
        if player_rect.right > screen.get_width():
            player_rect.right = screen.get_width()

# Schleife Hauptprogramm
while True:
    # Schritt 4: Überprüfen, ob Nutzer eine Aktion durchgeführt hat
    for event in pygame.event.get():
        # Beenden bei [ESC] oder [X]
        if event.type==QUIT or (event.type==KEYDOWN and event.key==K_ESCAPE):
            pygame.quit()

    # Schritt 5: Spiellogik
    update()

    # Schritt 6a: Spielfeld löschen
    screen.fill("lightblue")

    # Schritt 6b: Spielfeld/figuren zeichnen
    screen.blit(floor_surf, floor_rect)
    screen.blit(player_surf, player_rect)
    screen.blit(blk_surf, blk_rect)

    # Fenster aktualisieren
    pygame.display.flip()
    clock.tick(FPS)

# Version 2: Mit Klassen -> Übersichtlicher!!

Die Version 1 ist bereits ohne viel Logik ziemlich unübersichtlich. Es wäre daher sinnvoll Klassen zu erstellen. Für den Spieler und das fallende Objekt ist es in Version 2 exemplarisch gemacht worden. Man könnte auch den Boden zusätzlich als Klasse anlegen - die Kollisionsabfrage würde dann später einfacher.

Die beiden Klassen Player und Block erhalten im Konstruktor das Aussehen (surface) als Übergabeparameter. Der Spieler zusätzlich die Startposition, beim Block wird die Position zufällig innerhalb der Klasse ermittelt.
Beide Klassen haben zusätzlich eine eigene update-Methode, die von der bisherigen update-Methode dann aufgerufen wird. So bleibt die Logik jeweils beim Spielelement.

In [ ]:
import pygame
from pygame.locals import *
import random

class Player():
    def __init__(self, surface, position):
        self.rect = pygame.Surface.get_rect(surface)
        self.rect.midbottom = position
    
    def update(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.rect.x -= 3
            if self.rect.left < 0:
                self.rect.left = 0
        if keys[pygame.K_RIGHT]:
            self.rect.x += 3
            if self.rect.right > screen.get_width():
                self.rect.right = screen.get_width()

class Block():
    def __init__(self, surface):
        self.rect = pygame.Surface.get_rect(surface)
        self.rect.center = (random.randint(10,screen.get_width()-10), -30)

    def update(self):
        self.rect.y += 3
        if self.rect.bottom >= floor_rect.top:
            self.rect.center = (random.randint(10,screen.get_width()-10), -30)
    

In [ ]:
# Schritt 1: Importieren u. initialisieren der Pygame-Bibliothek
import random
import pygame
from pygame.locals import *
pygame.init()

# Schritt 2a: Variablen/KONSTANTEN setzen
SCREEN_WIDTH  = 300
SCREEN_HEIGHT = 480
FPS  = 60

# Schritt 2b: Hilfs-Funktionen für das Hauptprogramm

# Schritt 3: Definieren und Öffnen eines neuen Fensters
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Titel für Fensterkopf")
clock = pygame.time.Clock()

# Floor
floor_surf = pygame.Surface((screen.get_width(), screen.get_height()//4))
floor_surf.fill("green")
floor_rect = pygame.Surface.get_rect(floor_surf)
floor_rect.topleft = (0,screen.get_height()*3//4)

# Player
player_surf = pygame.Surface((20,20))
player_surf.fill("black")
#player_surf = pygame.image.load("images/player.png").convert_alpha()
player = Player(player_surf, floor_rect.midtop)  # <- Anlegen des Spielerobjekts

# Objects
blk_surf = pygame.Surface((20,20))
blk_surf.fill("red")
blk = Block(blk_surf)                            # <- Anlegen des Blockobjekts

def update():
    blk.update()
    player.update()
    
# Schleife Hauptprogramm
while True:
    # Schritt 4: Überprüfen, ob Nutzer eine Aktion durchgeführt hat
    for event in pygame.event.get():
        # Beenden bei [ESC] oder [X]
        if event.type==QUIT or (event.type==KEYDOWN and event.key==K_ESCAPE):
            pygame.quit()

    # Schritt 5: Spiellogik
    update()

    # Schritt 6a: Spielfeld löschen
    screen.fill("lightblue")

    # Schritt 6b: Spielfeld/figuren zeichnen
    screen.blit(floor_surf, floor_rect)
    screen.blit(player_surf, player.rect)
    screen.blit(blk_surf, blk.rect)

    # Fenster aktualisieren
    pygame.display.flip()
    clock.tick(FPS)

# Version 3: Kollisionsabfrage + Score:

Nun soll ein einfaches Score-System hinzugefügt werden: Immer, wenn der Spieler einen Block "auffängt", soll ein Zähler hochgezählt werden. Der Block soll dann nicht weiter bis zum Boden fallen und direkt wieder oben (außerhalb des sichtbaren Bereichs) auftauchen.

Für Score wird eine eigene Klasse verwendet.

Die Kollisionsabfrage findet in der Game-Loop statt: Da sowohl der Block als auch der Spieler ein Rect-Objekt haben, kann die Kollisionsabfrage sehr einfach über die Methode ```pygame.Rect.colliderect( rect1, rect2 )``` erfolgen. Diese liefert ```True``` zurück, wenn es eine Überlappung zweier Rect-Objekte gibt:

```collide = pygame.Rect.colliderect(player.rect, blk.rect)```

Spätestens jetzt sollte klar sein, warum es sinnvoll ist, die Spielelemente als Rect-Objekte zu behandeln. (Überlege Dir mal, wie viele Überprüfungen von Dir programmiert werden müssten, um festzustellen, ob sich zwei beliebig breite Rechtecke mit den Werte (x,y,breite,hoehe) überlappen?!!!)

In [ ]:
import pygame
from pygame.locals import *
import random

class Player():
    def __init__(self, surface, position):
        self.rect = pygame.Surface.get_rect(surface)
        self.rect.midbottom = position
    
    def update(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.rect.x -= 3
            if self.rect.left < 0:
                self.rect.left = 0
        if keys[pygame.K_RIGHT]:
            self.rect.x += 3
            if self.rect.right > screen.get_width():
                self.rect.right = screen.get_width()

class Block():
    def __init__(self, surface):
        self.rect = pygame.Surface.get_rect(surface)
        self.rect.center = (random.randint(10,screen.get_width()-10), -30)

    def update(self):
        self.rect.y += 3
        if self.rect.bottom >= floor_rect.top:
            self.respawn()
    
    def respawn(self):
        self.rect.center = (random.randint(10,screen.get_width()-10), -30)
    
class Score():
    def __init__(self):
        self.score = 0
        self.font = pygame.font.SysFont('Arial', 10, True, False) # Fett = True, kursiv = False

    def increase_score(self):
        self.score += 1
        
    def display(self):
        self.text = self.font.render(f"Score: {self.score}", True, "red")   # Anti-aliasing = True
        screen.blit(self.text, (0, 0))

In [ ]:
# Schritt 1: Importieren u. initialisieren der Pygame-Bibliothek
import random
import pygame
from pygame.locals import *
pygame.init()

# Schritt 2a: Variablen/KONSTANTEN setzen
SCREEN_WIDTH  = 300
SCREEN_HEIGHT = 480
FPS  = 60

# Schritt 2b: Hilfs-Funktionen für das Hauptprogramm

# Schritt 3: Definieren und Öffnen eines neuen Fensters
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Titel für Fensterkopf")
clock = pygame.time.Clock()

# Score
score = Score()

# Floor
floor_surf = pygame.Surface((screen.get_width(), screen.get_height()//4))
floor_surf.fill("green")
floor_rect = pygame.Surface.get_rect(floor_surf)
floor_rect.topleft = (0,screen.get_height()*3//4)

# Player
player_surf = pygame.Surface((20,20))
player_surf.fill("black")
#player_surf = pygame.image.load("images/player.png").convert_alpha()
player = Player(player_surf, floor_rect.midtop)

# Objects
blk_surf = pygame.Surface((20,20))
blk_surf.fill("red")
blk = Block(blk_surf)

def update():
    blk.update()
    player.update()
    
    collide = pygame.Rect.colliderect(player.rect, blk.rect)
   
    if collide:
        score.increase_score()
        blk.respawn()
        
# Schleife Hauptprogramm
while True:
    # Schritt 4: Überprüfen, ob Nutzer eine Aktion durchgeführt hat
    for event in pygame.event.get():
        # Beenden bei [ESC] oder [X]
        if event.type==QUIT or (event.type==KEYDOWN and event.key==K_ESCAPE):
            pygame.quit()

    # Schritt 5: Spiellogik
    update()

    # Schritt 6a: Spielfeld löschen
    screen.fill("lightblue")

    # Schritt 6b: Spielfeld/figuren zeichnen
    screen.blit(floor_surf, floor_rect)
    screen.blit(player_surf, player.rect)
    screen.blit(blk_surf, blk.rect)
    score.display()

    # Fenster aktualisieren
    pygame.display.flip()
    clock.tick(FPS)

# Version 4: Verwendung der Oberklasse 'Sprite':

Solange es nur zwei Objekte gibt, die sich bewegen und berühren können, ist das Konzept von Rect und Surface vollkommen ausreichend. 
Sollen aber mehrere Objekte überprüft werden, kann es sinnvoll sein, sich mit den pyGame-Klassen ```Sprite``` und ```(Sprite-)Group``` zu beschäftigen:

Stell Dir vor, ein Raumschiff gibt mehrere Schüsse hintereinander ab. Diese bewegen sich gleichzeitig über den Bildschirm. Triff ein Schuss ein anderes Objekt (z.B. einen von mehreren Gegnern), so soll dieser Gegner gelöscht werden. Die anderen Schüsse fliegen weiter und verschwinden erst dann, wenn sie z.B. auf eine Wand treffen.

Dann muss man die Schüsse entweder
* alle einzeln verwalten (sehr viel eigener Aufwand!!) oder
* in einer Liste verwalten (viel eigener Aufwand) oder
* kann diese in einer Sprite-Gruppe verwalten (wenig Aufwand).

Das nachfolgende Programm ist nun eine Variante vom bisherigen Spiel: 5 Blöcke fallen von oben herunter und können vom Spieler aufgefangen werden.

Da die Kollisionen von mehreren Blöcken wichtig sind, wurde diese Block-Klasse von der Klasse Sprite (pygame.sprite.Sprite) abgeleitet, d.h. sie erben von der Oberklasse pygame.sprite.Sprite: 

Damit man die Klasse Block nach der Vererbung von der Oberklasse pygame.sprite.Sprite benutzen kann, **muss** diese die Attribute ```self.rect``` und ```self.image``` enthalten. ```self.rect``` hatten wir bislang schon im Code, ```self.image``` kann man einfach die surface-Information zuweisen.
Außerdem **muss** man im Konstruktor den Konstruktor der Oberklasse aufrufen:
```pygame.sprite.Sprite.__init__(self)```

Die Methode ```update(...)``` sollte auch **genauso** heißen, denn diese wird später für jedes Sprite-Objekt mit genau diesem Namen aufgerufen werden!

Damit die Blöcke später als Gruppe verwaltet werden können, wird für diese eine Gruppe erstellt:

```blk_group = pygame.sprite.Group()```

Über ```blk_group.add(...)``` kann man dann beliebig viele Block-Objekte dieser Gruppe hinzuweisen und mit ```blk_group.remove(...)``` auch wieder entfernen. (Man kann auch einfach ein Objekt mit ```kill()``` löschen, dann wird es auch aus der Gruppe entfernt.)

Die Kollisionsabfrage von allen(!) Blöcken mit dem Spieler geht nun sehr einfach mit 

```collide_blk_list = pygame.sprite.spritecollide(player, blk_group, False)```

Der Rückgabewert ist eine Liste aller Blöcke, die mit dem Spieler eine Kollision haben. Diese Liste kann man auswerten und dann die Aktionen für den Block auslösen: Respawn und Score erhöhen.

Beachte: 
* Man muss die update(...)-Methode nur einmalig für die Block-**Gruppe** aufrufen: ```blk_group.update()``` -> Bei allen(!) Block-Objekten wird dann die update(...)-Methode der Block-Klasse ausgeführt. (Deshalb muss die Methode innerhalb der Block-Klasse auch unbedingt update(...) heißen.
* Auch das Zeichnen der Block-Objekte erfolgt über einen einzigen Aufruf von ```blk_group.draw(screen)```. Die Methode draw(...) muss nicht selbst erstellt werden.


In [ ]:
import pygame
from pygame.locals import *
import random

class Player():
    def __init__(self, surface, position):
        self.rect = pygame.Surface.get_rect(surface)
        self.rect.midbottom = position
    
    def update(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.rect.x -= 3
            if self.rect.left < 0:
                self.rect.left = 0
        if keys[pygame.K_RIGHT]:
            self.rect.x += 3
            if self.rect.right > screen.get_width():
                self.rect.right = screen.get_width()

class Block(pygame.sprite.Sprite):
    def __init__(self, surface):
        pygame.sprite.Sprite.__init__(self)
        self.image = surface
        self.rect = pygame.Surface.get_rect(surface)
        self.rect.center = (random.randint(10,screen.get_width()-10), -30)

    def update(self):
        self.rect.y += 3
        if self.rect.bottom >= floor_rect.top:
            self.respawn()
    
    def respawn(self):
        self.rect.center = (random.randint(10,screen.get_width()-10), -30)
    
class Score():
    def __init__(self):
        self.score = 0
        self.font = pygame.font.SysFont('Arial', 10, True, False) # Fett = True, kursiv = False

    def increase_score(self):
        self.score += 1
        
    def display(self):
        self.text = self.font.render(f"Score: {self.score}", True, "red")   # Anti-aliasing = True
        screen.blit(self.text, (0, 0))

In [ ]:
# Schritt 1: Importieren u. initialisieren der Pygame-Bibliothek
import random
import pygame
from pygame.locals import *
pygame.init()

# Schritt 2a: Variablen/KONSTANTEN setzen
SCREEN_WIDTH  = 300
SCREEN_HEIGHT = 480
FPS  = 60

# Schritt 2b: Hilfs-Funktionen für das Hauptprogramm

# Schritt 3: Definieren und Öffnen eines neuen Fensters
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Titel für Fensterkopf")
clock = pygame.time.Clock()

# Score
score = Score()

# Floor
floor_surf = pygame.Surface((screen.get_width(), screen.get_height()//4))
floor_surf.fill("green")
floor_rect = pygame.Surface.get_rect(floor_surf)
floor_rect.topleft = (0,screen.get_height()*3//4)

# Player
player_surf = pygame.Surface((20,20))
player_surf.fill("black")
#player_surf = pygame.image.load("images/player.png").convert_alpha()
player = Player(player_surf, floor_rect.midtop)

# Objects
blk_surf = pygame.Surface((20,20))
blk_surf.fill("red")
#blk_rect = pygame.Surface.get_rect(blk_surf)

blk_group = pygame.sprite.Group()           # <- Gruppe erstellen
for i in range(5):
    new_blk = Block(blk_surf)               # <- Neuen Block erstellen...
    blk_group.add(new_blk)                  #    ... und der Gruppe hinzufügen.

def update():
    blk_group.update()                      # <- Für die ganze Gruppe wird einzeln(!) die update()-Methode ausgelöst
    player.update()
    
    collide_blk_list = pygame.sprite.spritecollide(player, blk_group, False)
    
    if collide_blk_list:
        for blk in collide_blk_list:
            score.increase_score()
            blk.respawn()
            #blk.kill()
        
# Schleife Hauptprogramm
while True:
    # Schritt 4: Überprüfen, ob Nutzer eine Aktion durchgeführt hat
    for event in pygame.event.get():
        # Beenden bei [ESC] oder [X]
        if event.type==QUIT or (event.type==KEYDOWN and event.key==K_ESCAPE):
            pygame.quit()

    # Schritt 5: Spiellogik
    update()

    # Schritt 6a: Spielfeld löschen
    screen.fill("lightblue")

    # Schritt 6b: Spielfeld/figuren zeichnen
    screen.blit(floor_surf, floor_rect)
    screen.blit(player_surf, player.rect)
    blk_group.draw(screen)                   # <- Die ganze Gruppe wird gezeichnet
    score.display()

    # Fenster aktualisieren
    pygame.display.flip()
    clock.tick(FPS)